In [1]:
import torch
from model.vae import VAE_denoiser
from model.loader_utils import *

/hpcnfs/home/ieo5776/miniconda3/envs/pytorch/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


# CREATE THE FUNCTION TO RESHAPE DATA AND SAVE IT EXTERNALLY

In [2]:
VAE = VAE_denoiser()
VAE

VAE_denoiser(
  (enc_layer1): Linear(in_features=128, out_features=64, bias=True)
  (enc_layer2_mu): Linear(in_features=64, out_features=10, bias=True)
  (enc_layer2_logvar): Linear(in_features=64, out_features=10, bias=True)
  (dec_layer1): Linear(in_features=10, out_features=64, bias=True)
  (dec_layer2): Linear(in_features=64, out_features=128, bias=True)
)

In [3]:
train_dataset = BigwWigdDataset(input_dir='/hpcnfs/scratch/DP/amariani/Amariani/denoising_ChIPseq/data/tf', 
								bg_file='chrom1.bedGraph', 	Train=True)
test_dataset = BigwWigdDataset(input_dir='/hpcnfs/scratch/DP/amariani/Amariani/denoising_ChIPseq/data/tf', 
								bg_file='chrom1.bedGraph', 	Train=False)

train_DT = DataLoader(train_dataset, batch_size=128, shuffle=False, drop_last=True)
test_DT = DataLoader(train_dataset, batch_size=128, shuffle=False, drop_last=True)

In [12]:
train_dataset._get_data(True)

,chrom,start,end,score


In [ ]:
# initialize loss function and optimizer
adam = torch.optim.Adam(VAE.parameters(), lr=1e-3)

trained_model = train_model(train_DT=train_DT, 
							model=VAE,
							optimizer=adam,
							num_epochs=2)

#torch.save(trained_model, '/hpcnfs/scratch/DP/amariani/Amariani/denoising_ChIPseq/VAE_model.pth')

# TESTING

In [2]:
import pandas as pd

In [1]:
# __init__ would take an argument Train, if true yields train + validation, if false, yields test

# hardcoded chromosomes
test = ['chr1', 'chr2']
validation = ['chr3', 'chr4']
to_remove = ['chrY', 'chrM']

lists = [test, validation, to_remove]
lists

ring = pd.read_csv('/hpcnfs/scratch/DP/amariani/Amariani/denoising_ChIPseq/data/tf/RING1B.bedGraph', 
						sep='\t', names=['chrom', 'start', 'end', 'score'])

mask_test = ring['chrom'].isin(lists[0])
mask_validation = ring['chrom'].isin(lists[1])
mask_remove = ring['chrom'].isin(lists[2])

NameError: name 'pd' is not defined

In [13]:
lists[2]

['chrY', 'chrM']

In [11]:
ring['chrom'].value_counts()

chrom
chr1     1112044
chr2     1029173
chr3      918840
chr4      851438
chrX      838680
chr6      834620
chr5      833770
chr10     744083
chr7      728729
chr8      711018
chr11     694219
chr9      693277
chr13     669700
chr14     659606
chr12     644137
chr15     568963
chr16     561569
chr17     517916
chr18     507764
chr19     342007
chrY         399
chrM         288
Name: count, dtype: int64

In [ ]:
# # make this into a function
# bw = pyBigWig.open("/hpcnfs/scratch/DP/amariani/Samato/P124_project/data/ChIPseq_1/results/06bigwig/noSubtract/CTR-VILL-RING1B-15d_NoSpike.bw")
# bw_write = pyBigWig.open("/hpcnfs/scratch/DP/amariani/Amariani/denoising_ChIPseq/data/tf/chrom1.bw", "w")

# # this info needs to go the newly created BigWig file as the header
# # write the header, for now only chrom1
# len_chrm1 = bw.chroms()['chr1']
# bw_write.addHeader([('chr1', len_chrm1)])
# # add entries, create numpy coordinates
# chroms = np.array(["chr1"] * len(bw.intervals('chr1')))
# start = np.array([start[0] for start in bw.intervals('chr1')], dtype=np.int64)
# end = np.array([end[1] for end in bw.intervals('chr1')], dtype=np.int64)
# value = np.array([value[2] for value in bw.intervals('chr1')], dtype=np.float32)
# bw_write.addEntries(chroms, start, ends=end, values = value)
# bw_write.close()